In [2]:
#import os, os.path
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, regexp_tokenize
import pandas as pd
from collections import Counter
import numpy as np
import heapq
import ast
import re
import math

# functions is the file with our reading vocabulary, inv_indx and inv_indx_tfidf
import functions

These are all paths used in notebook.

In [3]:
path_all_files = r'C:\Users\alice\Desktop\ADM_HW3\file_unique'
path_all_files_doc = r'C:\Users\alice\Desktop\ADM_HW3\file_unique\doc_'
path_vocabulary = r'C:\Users\alice\Desktop\ADM_HW3\vocabulary.txt'
path_inverted_indx = r'C:\Users\alice\Desktop\ADM_HW3\inverted_indx.txt'
path_inv_indx_tfid = r'C:\Users\alice\Desktop\ADM_HW3\inverted_indx_tfid.txt'

# Search Engine 1

The **preprocess** function converts words in files.
* removing '\n'
* removing punctuation
* filter the non stopwords
* removing the stem

In [4]:
def preprocess(text):
    # converting to lower case
    text = text.lower()
    # removing '\n'
    text = text.replace('\\n', ' ')
    #----------------------------------------------------------------
    tokenizer = regexp_tokenize(text, "[\w\$]+")
    # filter the non stopwords
    filtered = [w for w in tokenizer if not w in stopwords.words('english')]
    ps = PorterStemmer()
    # removing the stemming
    filtered = [ps.stem(word) for word in filtered]
    # it returns tokenized text
    return filtered

The **findTheBestDocuments** is a function which is searching for documents with the all words from a query. 

In [5]:
def findTheBestDocuments(docs_list, pre_query):
    # it counts how many times each word of the query occurs in the documents
    # at first we initialize the words_dict we are adding the whole words from the first file with value 1
    words_dict = {word:1 for word in docs_list[0]}
    # now we are counting how many times each word occurs in the next files
    for sublist_ind in range(1, len(docs_list)):
        for k in docs_list[sublist_ind]:
            try:
                # if the word already exists
                words_dict[k] += 1
            except:
                # if the word doesn't exist
                words_dict[k] = 1
    # it returns only the documents which contain all the query words
    return [doc_id for doc_id in words_dict if words_dict[doc_id] == len(pre_query)]

**SearchEngine** is a function with the arguments:
* query - The input text from user
* vocab - dictionary (saved in 'vocabulary.txt' file)
* inv_indx - inverted index dictionary (saved in 'inv_ind.txt' file)

The **output** is a list of doc_id's for the best documents. 

In [6]:
def SearchEngine(query, vocabulary, inv_indx):
    pre_query = preprocess(query)
    word_list = []
    for item in pre_query:
        if item not in vocabulary:
            print('No documents found')
            break
        word_id = vocabulary[item]
        word_list.append(word_id)
    # word_list contains the id's of words according to vocabulary file
    
    result_list = []
    for term_id in word_list:
        result_list.append(inv_indx[term_id])
    # result_list contains the id's of documents which contain at least one word from the query
    
    best_docs = findTheBestDocuments(result_list, pre_query)
    # best_docs is the list of the id's of documents which contain all the words of the query
    
    return best_docs

The function that opens and reads the *vocabulary.txt* is in the **functions.py** script. We recall it and save the vocabulary as dictionary.

In [7]:
vocabulary = functions.read_vocabulary('vocabulary.txt')

We recall the function that opens the *inverted_indx.txt* and save the inv_index dictionary.

In [8]:
inv_indx = functions.read_inv_indx('inverted_indx.txt')

**Test of the SearchEngine:**

Take the user query as input.

In [8]:
query = input()

beautiful house


In [13]:
test = SearchEngine(query, vocabulary, inv_indx)

We use *pandas* to show the best documents for the user query so we obtain a table in which every row is a document with the whole informations needed.

In [10]:
docs_files = []
for i in test:
    docs_files.append(pd.read_csv(path_all_files_doc + str(i) + '.tsv', sep = '\t'))
docs_list = [[row for row in doc_i] for doc_i in docs_files]
cols = ['1', '2', 'City', '4', 'Description', '6', '7', 'Title', 'Url']
pd.set_option('max_colwidth', 500)
pd.DataFrame(docs_list, columns = cols)[['Title', 'Description', 'City', 'Url']]

,Title,Description,City,Url
0,Unique Location! Alamo Heights - Designer Inspired,"Stylish, fully remodeled home in upscale NW – Alamo Heights Area. \n\nAmazing location - House conveniently located in quiet street, with beautiful seasoned trees, prestigious neighborhood and very close to the airport, 281, 410 loop and down-town area. \n\nFeaturing an open floor plan, original hardwood floors, 3 bedrooms, 3 FULL bathrooms + an independent garden-TV room which can sleep 2 more\n\nEuropean inspired kitchen and “top of the line” decor. Driveway can park 4 cars.",San Antonio,https://www.airbnb.com/rooms/17481455?location=Cibolo%2C%20TX\r\n
1,Friendly Private Room in َQuiet Neighborhood,"This is a beautiful bedroom with a queen size bed and closet. We do not have pets and the house is always clean. The bathroom is shared and supplies such as towels and shampoo are available. We are only some miles from Downtown, TCU, TCC, and Stockyards.",Fort Worth,https://www.airbnb.com/rooms/18977363?location=Cleburne%2C%20TX\r\n
2,Bethel Blue,Bethel Blue is a cozy 2 bedroom 1.5 bath house about 5 blocks from the beautiful beach. It is decorated nicely and equipped with everything your family needs for a wonderful beach getaway. Military and Senior citizens receive a 10% discount!,Bolivar Peninsula,https://www.airbnb.com/rooms/12253684?location=Bolivar%20Peninsula%2C%20TX\r\n
3,A Cozy Home with a Beautiful Nature Views,"This home is on the North Side of San Antonio, 3 minutes away from a great golfing course and a JW Marriott Resort and Spa. This is a great stopping place for a family and comes with all amenities from cable, internet to a laundry room and 2 bathrooms. This house is against a green belt so you can enjoy the back yard and nature.This area is safe and my neighbors are also great! Hope you enjoy your stay and more so enjoy San Antonio!",San Antonio,https://www.airbnb.com/rooms/19190311?location=Cibolo%2C%20TX\r\n
4,Cabins on the Square House,"Have the perfect little getaway at the quaint and cozy Cabins on the Square. The location is perfect! Located in the middle of the shopping square and on the beautiful rolling river in downtown Wimberley, TX. You are in walking distance to EVERYTHING. There are countless quirky finds in all of the local shops. You never know what you will find! After you are all done walking, shopping and nibbling at the local hot spots, come sit right outside your cabin on the bank of the Blanco River.",Wimberley,https://www.airbnb.com/rooms/18043874?location=Canyon%20Lake%2C%20TX\r\n
5,Monroe Manor,Come relax on the water in our beautiful beach house style river home. You'll love our shiplap walls but the view is even better!,Houston,https://www.airbnb.com/rooms/19273801?location=Channelview%2C%20TX\r\n
6,Grand Oak Guest House.,"Take a trip away from the bustle of the city and into the heart of some of the largest oak trees in Texas for a relaxing and tranquil vacation. Located across from Miller Creek in Johnson City, Texas, this farmhouse property is nestled amongst some of the most current and adorable homes in the Texas Hill Country. Minutes from popular, award-winning vineyards, numerous restaurants, and the enchanting Pedernales Falls State Park, this area has it all on the beautiful Whittington Ranch.",Johnson City,https://www.airbnb.com/rooms/13556577?location=Blanco%2C%20TX\r\n
7,Little Blanco,"Little Blanco is a gorgeous vacation rental house located in Concan, Texas. It is in the beautiful 18 hole golf course in Concan. It is also on one of the highest points of the Golf Course. It has over 4342 sq. ft. of livable space for you and your guests. It has 4 bedrooms and 4 bathrooms and sleeps 28 people. It is a single level home.",Concan,https://www.airbnb.com/rooms/17808720?location=Concan%2C%20TX\r\n
8,Warm & friendly B & B in S. Austin!,"The private bedroom is painted in soothing grays, decorated with colorful pictures and has a real Queen bed dressed in bright linens. There is a spacious walk-in clo

# Search Engine 2

We recall the function that opens the 'inverted_indx_tfid.txt' and save the inv_idx_tfidf dictionary.

In [9]:
inv_indx_tfidf = functions.read_inv_indx_tfidf(file = 'inverted_indx_tfid.txt')

We use **numpy** library to compute cosine distance between two vectors.

In [10]:
def cosine_dist(x, y):
    dot_product = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    return 1 - (dot_product / (norm_x * norm_y))

The function **query_tfidf** calculates the TFIDF score for words in a query and returns the occurrence of each word in the query:

In [11]:
def query_tfidf(prep_query_list):
    return [prep_query_list.count(word) for word in prep_query_list]

The function **get_tfidf** returns the TFIDF score for a word in the (doc_id) document.

ARGS:
* arg_list - it is the value (as a list) for a given word (the key of the dict) in 'inv_indx_tfid' dictionary.
* doc_id - the document id for which function returns the tfid score

In [12]:
def get_tfidf(arg_list, doc_id):
    for tuple_ in arg_list:
        # tuple format: (doc_id, tdidf_score)
        if tuple_[0] == doc_id:
            return tuple_[1]

The function **get_top_n** returns n documents sorted by cosine similarity. We use the **heap** structure: actually we work on the **cos_dist** and get the top n_smallest in order to have the top of n_biggest **cos_similarity** which is $1-cos\_dist$.

In [13]:
def get_top_n(n, score_doc_list):
    heap = []
    for tup in score_doc_list:
        # creating the heap structure
         heapq.heappush(heap, tup)
    return heapq.nsmallest(n, heap)

**SearchEngine_new** is a function with the arguments:

* query - The input text from user
* vocabulary - dictionary (saved in 'vocabulary.txt' file)
* inv_indx - inverted index dictionary (saved in 'inv_ind.txt' file)
* inv_indx_tfidf  - inverted index with the tfidf score dictionary (saved in 'inv_ind_tfid.txt' file)
* n  - number of documents we want as return

The output is the dict of the n best documents with the corresponding cosine distances.

In [18]:
def SearchEngine_new(query, vocabulary, inv_indx, inv_indx_tfidf, n = 20):
    query_list = preprocess(query)
    word_list = []
    for item in query_list:
        if item not in vocabulary:
            print('No documents found')
            break
        word_id = vocabulary[item]
        word_list.append(word_id)
        
    result_list = []
    for term_id in word_list:
        result_list.append(inv_indx[term_id])
    selected_docs = findTheBestDocuments(result_list, query_list)
    
    # calculating the Cosine Similarities
    cos_dist_list = []
    for doc_id in selected_docs:
        
        # creating the TFID vector for a document
        tfid_vector = []
        
        for word in word_list: 
            g = get_tfidf(inv_indx_tfidf[word], doc_id)
            tfid_vector.append(g)    
        # creating the list of cos_dist and doc_id
        cos_dist_list.append((round(cosine_dist(query_tfidf(word_list), tfid_vector), 6), doc_id))
        # selecting the n best documents 
        top_n_list = get_top_n(n, cos_dist_list)
        
    return top_n_list

**Test of the SearchEngine_new:**

In [16]:
query = input()

a beautiful house


In [19]:
top_n_list = SearchEngine_new(query, vocabulary, inv_indx, inv_indx_tfidf)

This function creates the table with the whole information from selected files sorted by the **cosine similarity**:

In [20]:
def display_df_with_cos(top_n_list, path_all_files_doc):
    cos_dist = [(1-k, v) for k, v in top_n_list]
    cos_dataframe = pd.DataFrame(cos_dist)
    cos_dataframe.columns = ['Similarity', 'Doc_id']
    cos_dataframe_values = [v for k, v in cos_dist]
    docs_files = []
    for i in cos_dataframe_values:
        docs_files.append(pd.read_csv(path_all_files_doc + str(i) + '.tsv', sep = '\t'))
    docs_list = [[row for row in doc_i] for doc_i in docs_files]
    cols = ['1', '2', 'City', '4','Description', '6', '7', 'Title', 'Url']

    z = pd.DataFrame(docs_list, columns = cols)[['Title', 'Description', 'City', 'Url']]
    df = pd.concat([z, cos_dataframe], axis = 1)
    
    # we don't want to show the 'Doc_id' column
    df.drop(columns = ['Doc_id'], inplace = True)
    
    return df.style.set_table_styles([{'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}])

In [33]:
display_df_with_cos(top_n_list, path_all_files_doc)

,Title,Description,City,Url,Similarity
0,Beautiful Creekside Country Home,"Edelweiss is a lovely country home on the banks of beautiful Smith Creek, yet only minutes from the fabulous shopping and dining of downtown Wimberley. This spacious house is perfect for family and friend gatherings. Take a stroll through the 12 acres of creek side nature (2 private, 10 shared with the owners), watch the deer graze as you enjoy your morning coffee, strum your guitar beside the fire pit, challenge your friends to a game of pool, watch your kiddos play in the tree house, have a family bbq on the gorgeous back deck, snuggle up by the fireplace with a cup of hot cocoa, and feel the stress of your daily life melt away.\n\nThe name Edelweiss belongs to a delicate white flower that grows in the Alps, made famous in The Sound of Music. From the morning songs of the sweet, chirping birds to the evening orchestra courtesy of the frogs and toads along the creek, we like to say that the Hill Country is alive with the sound of music! Here you get the privacy and serenity of a country home with all the amenities of modern life, including wifi. During the house remodel the kitchen was built from scratch, and some of the furniture was hand made by the owner, a professional woodworker. The house has two and a half baths, three bedrooms (king, queen, and three twin beds), plus a large game room with two queen futons, games for kids of all ages, and a pool table. Add that to the spacious living room with a fold-out couch, dining room with a table for 8, and the large deck with a beautiful hand-made picnic table, and you’ve got the perfect space for your family and friend retreat! At Edelweiss the owners are environmentally conscious. They offer full recycling, use environmentally friendly dish soap, provide recycled paper products, offer a reverse-osmosis filtration system to eliminate the need for bottled water, and landscape minimally with native low-water plants. \n\nIf you plan to visit in the summer with children, you might also be interested in enrolling in one of the owner’s fun art classes next door. Jennifer, a TX certified art teacher, offers the best creative summer camps for small, peaceful groups in her creek side studio. Check out her offerings for kids ages 6+ at Agua Fresca Studios. \n\nWe are confident that Edelweiss will exceed your expectations. Just read some of the comments from the previous guests. The owners really go out of their way to add those special touches to make your trip complete, such as keeping the kitchen stocked with condiments and basics that you may have forgotten. When their chickens can keep up you’ll even find fresh, organic eggs in the fridge. You won’t want to leave Edelweiss, but you can always come back for another visit!\n\n\nPet policy:\nPets allowed at $50 per pet and we ask that you leave cash or check for the pet fee with the keys upon checkout. Pets must be declared to the owners in advance, preferably at the time of booking. We ask that if dogs are left unattended at the home that they be crated and that they remain on leash when outside the fenced yard if they might be inclined to chase a cat, chicken or child. Thanks!\n\nThis home sits on two private acres and we allow our guests as much privacy as possible. We also invite guests to explore our 10 shared acres next door, and are happy to strike up a conversation in passing. We love meeting new people!\n\nThere is a lovely creek view from the backyard, but the bank is steeper here and access down to the water is not super easy. There are multiple easy creek access points on the shared 10 acres, though. While our creek is a year 'round creek, our extended Texas drought has taken its toll on area springs and water levels can fluctuate during hot, dry spells.",Wimberley,https://www.airbnb.com/rooms/434915?location=Canyon%20Lake%2C%20TX,0.998618
1,Canyon Lake Hideaway- 3 Relaxing Acres next to Potter's Creek Park!,"CANYON LAKE HIDEAWAY - a SkyRun Texas Pro

# New scoring Function

Textual query from the user:

In [35]:
query = input()

a beautiful house


We request additional informations in order to perform our research:

In [21]:
city_user = input('Please insert the city: ')
beds_user = int(input('Please insert the number of bedrooms: '))
price_user = int(input('Please insert the price: '))

Please insert the city: 
Please insert the number of bedrooms: 1
Please insert the price: 40


The **scoringFunction** calculates the new score for one document. We give the lowest score to the best 'match' because we use a heap structure with **nsmallest** function. We know that there are some missing or even wrong informations inside our documents so we use **try** and **except** in order to take them into account and give (them) a low score. 

We give a weight in range $(0,1)$ to each informations we have, in particular $0.65$ to the city_score, $0.25$ to the bed_score and $0.1$ to the price_score. For example we give $0$ as score when the city of the document is that required by the user and similarly for the bed_score and price_score. Furthermore we consider cases in which, for example, the difference between the document price and the user one is not so relevant (in our opinion) and give an intermediate score. So we get our weighted score.

In [22]:
def scoringFunction(city_user, beds_user, price_user, document_id):
    with open(path_all_files_doc + str(document_id) + '.tsv', 'r', encoding = 'utf8') as csvfile:
        file1 = csv.reader(csvfile, delimiter = '\t')
        
        # inside the price column we don't need the $ symbol
        remove_dol = re.compile(r'[^\d.,]+')
        
        # we read the documents (formatted by '\t') as columns and select those of the city, price and bedrooms number
        columns = [i for i in file1]
        
        city = columns[0][2].lower()
        if city == city_user: 
            city_score = 0
        else: city_score = 1
            
        try:
            beds = int(columns[0][1])
            if beds == int(beds_user): 
                beds_score = 0
            elif beds < beds_user:
                beds_score = 0.9
            else:
                beds_score = 0.1
        except:
            beds_score = 1
        
        try:
            price = int(remove_dol.sub('', columns[0][0]))
            if price == int(price_user): 
                price_score = 0
            elif price < price_user:
                price_score = 0.1
            elif price - price_user <= 10:
                price_score = 0.3
            else:
                price_score = 1
        except:
            price_score = 1
        
        tot_score = city_score * 0.65 + price_score * 0.1 + beds_score * 0.25
        
        return round(tot_score, 4)

The **sorted_documents** function returns the list of the top n documents (doc_id) sorted by score (from the highest one).

In [23]:
def sorted_documents(selected_documents, city_user, beds_user, price_user, n):
    sorted_list = []
    for doc_id in selected_documents:
        tot_score = scoringFunction(city_user, beds_user, price_user, doc_id)
        sorted_list.append((tot_score, doc_id))
    top_n_list = get_top_n(n, sorted_list)
    
    return top_n_list

In [24]:
selected_documents = SearchEngine(query, vocabulary, inv_indx)

In [26]:
top_n_list = sorted_documents(selected_documents, city_user.lower(), beds_user, price_user, n = 20)

# we are using heap function 'nsmallest' so we print (1-k) in order to have the highest score for the best document
top_n_list = [(1-k, v) for (k,v) in top_n_list]

In [27]:
def display_df_with_rank(top_n_list, path_all_files_doc):
    doc_id_list = [v for k, v in top_n_list]
    docs_files = []
    for i in doc_id_list:
        docs_files.append(pd.read_csv(path_all_files_doc + str(i) + '.tsv', sep = '\t'))
    docs_list = [[row for row in doc_i] for doc_i in docs_files]
    cols = ['1', '2', 'City', '4','Description', '6', '7', 'Title', 'Url']
    pd.set_option('max_colwidth', 500)
    z = pd.DataFrame(docs_list, columns = cols)[['Title', 'Description', 'City', 'Url']]

    z.insert(0, 'Ranking', z.index + 1)
    z.reset_index(drop=True)

    return z.style.set_table_styles([{'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}])

In [28]:
display_df_with_rank(top_n_list, path_all_files_doc)

,Ranking,Title,Description,City,Url
0,1,Spacious room- Dallas 20 min,"This beautifully renovated modern two-story house provides you with a wonderful place to relax after a long day of work, shopping, exploring Dallas, or spending time with family and friends. Clean, serene, romantic and convenient. Perfect for couples, business travelers or lone wanderers.",Balch Springs,https://www.airbnb.com/rooms/16354120?location=Balch%20Springs%2C%20TX
1,2,Cozy room in a gorgeous house,"Beautiful 3 BDR, 2.5 bath home in a great neighborhood. NE location close to Randolph AFB and plenty of shopping and entertainments. Home features top of the line appliances, a game room with pool table and dart board, as well as movie theatre!",Schertz,https://www.airbnb.com/rooms/12450822?location=Cibolo%2C%20TX
2,3,Beautiful House Near Six Flags,"My sassy and chić house is tastefully decorated. You'll feel truly at home and welcomed! It comfortably fits two and is centrally located near La Cantera Mall, Top Golf, The Rim Shopping center, and Fiesta Texas. Be impressed by the hosts that feel like friends, and the accessibility that our house offers to all the major highways, dining, and fabulous entertainment!We look forward to hosting you!\n\nI'm pretty easy to work with so if you can convince that you'd be a great guest I'll discount you!",San Antonio,https://www.airbnb.com/rooms/18820560?location=Bulverde%2C%20TX
3,4,Comfortable Accommodation,A beautiful queen bed accomodation in a safe neighborhood. Dishwasher and dryer provided for your convenience. Video Security camera also installed outside the house for your safety.,Richmond,https://www.airbnb.com/rooms/14257375?location=Cinco%20Ranch%2C%20TX
4,5,Country Escape!,"We live in a 4 bedroom 2 bath home. We only occupy the master, which is on the opposite side of the house from our other 3 spare bedrooms and bath. We live in a beautiful country setting. We are 10 minutes to highway 20 and about twenty minutes from downtown. My wife and I are very clean and organized. We love having guests and meeting new people!",Fort Worth,https://www.airbnb.com/rooms/19037263?location=Benbrook%2C%20TX
5,6,Private room with Bathroom in South Austin,"No Cleaning or Any Additional Fees!\n\nBeautiful large modern home in South Austin, our house was recently built in a secured and safe community. Located only 15 miles away from Downtown Austin, blocks from I-35, world class restaurants and shopping.",Buda,https://www.airbnb.com/rooms/18255979?location=Buda%2C%20TX
6,7,Private room with Bathroom near Austin,"No Cleaning or Any Additional Fees!\n\nBeautiful large modern home in South Austin, our house was recently built in a secured and safe community. Located only 15 miles away from Downtown Austin, blocks from I-35, world class restaurants and shopping.",Buda,https://www.airbnb.com/rooms/17572156?location=Buda%2C%20TX
7,8,Nice cozy room - Downtown Dallas 20 min,"This beautifully renovated modern two-story house provides you with a wonderful place to relax after a long day of work, shopping, exploring Dallas, or spending time with family and friends. Clean, serene, romantic and convenient. Perfect for couples, business travelers or lone wanderers.",Balch Springs,https://www.airbnb.com/rooms/16353321?location=Balch%20Springs%2C%20TX
8,9,Friendly Private Room in َQuiet Neighborhood,"This is a beautiful bedroom with a queen size bed and closet. We do not have pets and the house is always clean. The bathroom is shared and supplies such as towels and shampoo are available. We are only some miles from Downtown, TCU, TCC, and Stockyards.",Fort Worth,https://www.airbnb.com/rooms/18977363?location=Cleburne%2C%20TX
9,10,Beautiful king bedroom in NW Austin,"My house is close to Lakeline Mall, highways and the toll road, restaurants, grocery stores, hike and bike trails, parks, Lake Travis. You’ll love my home because of the quiet, safe neighborhood, the light, the really comfy king size bed, the sunny kitchen, the gardens, hammocks